In [ ]:
# %%
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import (
    Input, Dense, Dropout, LayerNormalization, LSTM, RepeatVector,
    TimeDistributed, MultiHeadAttention, Concatenate, Bidirectional
)
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from keras_tuner import HyperParameters, RandomSearch

# Carregar dados
data = pd.read_csv("dataset.csv")

# Converter 'id' em datetime e definir como índice
data['timestamp'] = pd.to_datetime(data['id'], errors='coerce')
data.set_index('timestamp', inplace=True)


# Engenharia de Atributos
data['ws100_diff'] = data['ws100'].diff()
data['ws100_shift_1'] = data['ws100'].shift(1)
data['ws100_roll_mean_3'] = data['ws100'].rolling(window=3).mean()
data['ws100_roll_std_3'] = data['ws100'].rolling(window=3).std()
data['ws100_roll_mean_6'] = data['ws100'].rolling(window=6).mean()
data['ws100_roll_std_6'] = data['ws100'].rolling(window=6).std()

# Remover os NaNs gerados pelas operações
data = data.dropna()

# Selecionar variáveis
variables = data[['ws100', 'humid','wdisp100']].values

# Padronização dos dados
scaler = StandardScaler()
variables_scaled = scaler.fit_transform(variables)

# Parâmetros
sequence_length = 36
forecast_horizon = 6  # Não usado neste caso
split_ratio = 0.8

# Divisão dos dados em treinamento e teste
split_index = int(len(variables_scaled) * split_ratio)
train_data = variables_scaled[:split_index]
test_data = variables_scaled[split_index:]

# Função para preparar sequências de dados
def create_sequences(data, seq_length, target_step):
    X, y = [], []
    for i in range(len(data) - seq_length - target_step + 1):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length + target_step - 1, 0])  # 'ws100' no sexto passo futuro
    return np.array(X), np.array(y)

target_step = 6  # Prever o sexto valor futuro

# Criar sequências para treinamento e teste
X_train, y_train = create_sequences(train_data, sequence_length, target_step)
X_test, y_test = create_sequences(test_data, sequence_length, target_step)

# Reshape de y_train e y_test para serem compatíveis com a saída do modelo
y_train = y_train.reshape(-1, 1, 1)
y_test = y_test.reshape(-1, 1, 1)

# Obter média e desvio padrão de 'ws100' no conjunto de treinamento para inversão da padronização
mean_ws100 = scaler.mean_[0]
std_ws100 = np.sqrt(scaler.var_[0])

# Definição das funções de perda personalizadas
def custom_mse_sixth(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true[:, -1, 0] - y_pred[:, -1, 0]))

def custom_mae_sixth(y_true, y_pred):
    return tf.reduce_mean(tf.abs(y_true[:, -1, 0] - y_pred[:, -1, 0]))

def custom_rmse_sixth(y_true, y_pred):
    return tf.sqrt(custom_mse_sixth(y_true, y_pred))

def custom_mape_sixth(y_true, y_pred):
    epsilon = tf.keras.backend.epsilon()
    y_true_sixth = y_true[:, -1, 0]
    y_pred_sixth = y_pred[:, -1, 0]
    return tf.reduce_mean(tf.abs((y_true_sixth - y_pred_sixth) / (tf.abs(y_true_sixth) + epsilon))) * 100

# %%
# Definição da função para construir o modelo otimizado
def build_model_optimized(hp):
    # Hiperparâmetros com espaços de busca reduzidos
    units = hp.Int('units', min_value=128, max_value=256, step=32)
    learning_rate = hp.Choice('learning_rate', [1e-4, 1e-3])
    dropout_rate = hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1)
    num_layers = hp.Int('num_layers', min_value=1, max_value=2, step=1)
    optimizer_choice = hp.Choice('optimizer', ['adam', 'rmsprop'])
    
    # Seleção do Otimizador
    if optimizer_choice == 'adam':
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_choice == 'rmsprop':
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    
    # Encoder Input
    encoder_inputs = tf.keras.Input(shape=(sequence_length, X_train.shape[2]), name='encoder_input')
    encoder_outputs = encoder_inputs
    
    # Encoder Layers Bidirecionais
    for i in range(num_layers):
        encoder_outputs = Bidirectional(
            LSTM(
                units=units,
                return_sequences=True,
                activation='tanh',
                dropout=dropout_rate,
                kernel_regularizer=l2(1e-3),
                name=f'encoder_lstm_{i}'
            )
        )(encoder_outputs)
        encoder_outputs = LayerNormalization(name=f'encoder_norm_{i}')(encoder_outputs)
    
    # Encoder State (concatenado das direções forward e backward)
    encoder_state = encoder_outputs[:, -1, :]
    
    # Decoder Input
    decoder_inputs = RepeatVector(1, name='repeat_vector')(encoder_state)
    decoder_outputs = decoder_inputs
    
    # Decoder Layers Bidirecionais
    for i in range(num_layers):
        decoder_outputs = Bidirectional(
            LSTM(
                units=units,
                return_sequences=True,
                activation='tanh',
                dropout=dropout_rate,
                kernel_regularizer=l2(1e-3),
                name=f'decoder_lstm_{i}'
            )
        )(decoder_outputs)
        decoder_outputs = LayerNormalization(name=f'decoder_norm_{i}')(decoder_outputs)
    
    # Atenção Multi-Cabeça (adaptada para bidirecional)
    attention = MultiHeadAttention(
        num_heads=hp.Int('num_heads', min_value=2, max_value=4, step=2),
        key_dim=units*2,  # Dobrando as unidades devido à bidirecionalidade
        name='multi_head_attention'
    )(decoder_outputs, encoder_outputs)
    
    attention = Dropout(dropout_rate, name='attention_dropout')(attention)
    decoder_concat_input = Concatenate(axis=-1, name='concat_attention')([decoder_outputs, attention])
    
    # Camada de Saída
    outputs = TimeDistributed(Dense(1, activation='linear'), name='output_layer')(decoder_concat_input)
    
    # Definição do Modelo
    model = Model(inputs=encoder_inputs, outputs=outputs, name='Wind_Speed_Predictor_Bidirectional')
    
    # Compilação do Modelo
    model.compile(
        optimizer=optimizer,
        loss=custom_mse_sixth,  # Usando a classe de perda customizada
        metrics=[
            custom_mae_sixth,
            custom_rmse_sixth,
            custom_mape_sixth        
        ]
    )
    
    
    return model

# %%
# Otimização de hiperparâmetros com o Keras Tuner
tuner = RandomSearch(
    build_model_optimized,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    directory='tuner_dir',
    project_name='wind_speed_prediction_optimized'
)

# Early stopping para evitar overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

# Iniciar a busca de hiperparâmetros
tuner.search(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping]
)




Trial 5 Complete [00h 02m 29s]
val_loss: 0.5475764870643616

Best val_loss So Far: 0.5475764870643616
Total elapsed time: 00h 37m 22s

Search: Running Trial #6

Value             |Best Value So Far |Hyperparameter
192               |160               |units
0.0001            |0.001             |learning_rate
0.4               |0.3               |dropout_rate
2                 |1                 |num_layers
adam              |adam              |optimizer
2                 |2                 |num_heads

Epoch 1/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 24s 81ms/step - custom_mae_sixth: 0.9549 - custom_mape_sixth: 660.6164 - custom_rmse_sixth: 1.1942 - loss: 4.4526 - val_custom_mae_sixth: 0.8113 - val_custom_mape_sixth: 283.7258 - val_custom_rmse_sixth: 0.9911 - val_loss: 3.8469
Epoch 2/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 19s 99ms/step - custom_mae_sixth: 0.7753 - custom_mape_sixth: 390.6997 - custom_rmse_sixth: 0.9843 - loss: 3.7022 - val_custom_mae_sixth: 0.7703 - val_custom_mape_sixth: 253.6180 - va

In [ ]:
# %% Obter os Melhores Hiperparâmetros e o Melhor Modelo
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.get_best_models(num_models=1)[0]

# Não há necessidade de carregar pesos manualmente
# best_model.load_weights(best_trial.model_checkpoint_path)  # Remova esta linha

# %% Avaliação do Melhor Modelo
y_pred = best_model.predict(X_test)

# Inversão da padronização
y_pred_inv = y_pred[:, -1, 0] * std_ws100 + mean_ws100
y_test_inv = y_test[:, -1, 0] * std_ws100 + mean_ws100

# Métricas de desempenho
mae = mean_absolute_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
mape = np.mean(np.abs((y_test_inv - y_pred_inv) / y_test_inv)) * 100

print(f"MAE no sexto passo: {mae:.2f}")
print(f"RMSE no sexto passo: {rmse:.2f}")
print(f"MAPE no sexto passo: {mape:.2f}%")

# Plotagem das previsões
plt.figure(figsize=(12, 6))
plt.plot(y_test_inv, label='Valor Real - Passo 6')
plt.plot(y_pred_inv, label='Previsão - Passo 6')
plt.legend()
plt.title('Comparação entre Valores Reais e Previstos no Sexto Passo (Modelo Otimizado)')
plt.xlabel('Amostras')
plt.ylabel('Velocidade do Vento a 100 metros')
plt.show()

/home/lucas/Projetos/TFG-Vento-Sul/.venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 34 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


ValueError: A total of 11 objects could not be loaded. Example error message for object <LSTMCell name=lstm_cell, built=True>:

Layer 'lstm_cell' expected 3 variables, but received 0 variables during loading. Expected: ['kernel', 'recurrent_kernel', 'bias']

List of objects that could not be loaded:
[<LSTMCell name=lstm_cell, built=True>, <LSTMCell name=lstm_cell, built=True>, <LayerNormalization name=encoder_norm_0, built=True>, <LSTMCell name=lstm_cell, built=True>, <LSTMCell name=lstm_cell, built=True>, <LayerNormalization name=decoder_norm_0, built=True>, <EinsumDense name=key, built=True>, <EinsumDense name=attention_output, built=True>, <EinsumDense name=query, built=True>, <EinsumDense name=value, built=True>, <Dense name=dense, built=True>]